Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

In [636]:
def merged_dataframe(): #function that returns a merged dataframe 
    import pandas as pd
    import numpy as np
    energy = pd.read_excel('Energy Indicators.xls', header = None, skip_footer = 1)
    energy = energy.drop(energy.columns[[0,1]], axis = 1)
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy = energy[18:]
    energy['Energy Supply'][energy['Energy Supply'] == '...'] = np.nan
    energy['Energy Supply'] = energy['Energy Supply'] * 1000000
    energy['Energy Supply per Capita'][energy['Energy Supply per Capita'] == '...'] = np.nan
    energy['Country'] = energy['Country'].str.replace(r" \(.*\)","").str.replace('\d+','')
    energy['Country'] = energy['Country'].replace({"Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"})
    GDP = pd.read_csv('world_bank.csv', header = None, skiprows = 4)
    GDP[0] = GDP[0].replace({"Korea, Rep.": "South Korea", 
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"})
    a = np.arange(1,50,1).tolist()
    GDP = GDP.drop(GDP.columns[a], axis = 1)
    GDP.columns = ['Country','2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    GDP = GDP[1:]
    ScimEn = pd.read_excel('scimagojr-3.xlsx')
    ScimEn1 = ScimEn[0:15]
    preans = pd.merge(ScimEn1, energy, how = 'inner', left_on = 'Country', right_on = 'Country')
    ans = pd.merge(preans, GDP, how = 'inner', left_on = 'Country', right_on = 'Country')
    ans = ans.set_index('Country')
    return ans


In [603]:
def lost_entries(): #function that returns the number of lost entries in the merging
    import pandas as pd
    import numpy as np

    energy = pd.read_excel('Energy Indicators.xls', skiprows=17,skip_footer= 38)
    energy = energy[['Unnamed: 1','Petajoules','Gigajoules','%']]
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] =  energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].replace('...',np.NaN).apply(pd.to_numeric)
    energy['Energy Supply'] = energy['Energy Supply']*1000000
    energy['Country'] = energy['Country'].replace({'China, Hong Kong Special Administrative Region':'Hong Kong','United Kingdom of Great Britain and Northern Ireland':'United Kingdom','Republic of Korea':'South Korea','United States of America':'United States','Iran (Islamic Republic of)':'Iran'})
    energy['Country'] = energy['Country'].str.replace(" \(.*\)","")
    
    GDP = pd.read_csv('world_bank.csv',skiprows=4)
    GDP['Country Name'] = GDP['Country Name'].replace('Korea, Rep.','South Korea')
    GDP['Country Name'] = GDP['Country Name'].replace('Iran, Islamic Rep.','Iran')
    GDP['Country Name'] = GDP['Country Name'].replace('Hong Kong SAR, China','Hong Kong')
    columns = ['Country Name','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
    GDP = GDP[columns]
    GDP.columns = ['Country','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
    ScimEn = pd.read_excel('scimagojr-3.xlsx')
    ScimEn_m = ScimEn[:15]
    df = pd.merge(ScimEn, energy, how = 'inner', left_on = 'Country', right_on='Country')
    final_df = pd.merge(df,GDP, how = 'inner', left_on = 'Country', right_on='Country')
    final_df = final_df.set_index('Country')
    columns = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    ans = final_df[columns]
    df2 = pd.merge(ScimEn, energy, how = 'outer', left_on = 'Country', right_on='Country')
    final_df2 = pd.merge(df2,GDP, how = 'outer', left_on = 'Country', right_on='Country')
    ans2 = len(final_df2) - len(final_df)
    return ans2


156

In [604]:
def average_gdp(): #function that returns the average GDP over the last 10 years for each country
    Top15 = merged_dataframe()
    new_df = Top15.drop(Top15.columns[[0,1,2,3,4,5,6,7,8,9]], axis = 1)
    new_df['mean'] = new_df.mean(axis = 1).sort_values(ascending = False)
    new_df = new_df.sort(['mean'], ascending = False)
    avgGDP = new_df['mean']
    return avgGDP


/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: mean, dtype: float64

In [374]:
def GDP_change_UK(): #returns the GDP difference for UK in lat 10 years
    Top15 = merged_dataframe()
    GDP_change = Top15[Top15.index == 'United Kingdom']['2015'] - Top15[Top15.index == 'United Kingdom']['2006']
    return pd.to_numeric(GDP_change)[0]


246702696075.3999

In [605]:
def energy_suppy_per_capita(): #returns the energy supply per capita
    Top15 = merged_dataframe()
    mean_ESPC = Top15['Energy Supply per Capita'].mean()
    return mean_ESPC


157.59999999999999

In [606]:
def renewable_energy(): #Country with the maximum % renewable energy and the percentage
    Top15 = merged_dataframe()
    per = Top15['% Renewable'].max()
    name_country = (Top15[Top15['% Renewable'] == per].index).tolist()[0]
    return (str(name_country), per)


('Brazil', 69.64803)

In [607]:
def max_ratio_self_to_total(): #function returns the ratio of self-citations to toal citations in university of the country with highest ratio
    Top15 = merged_dataframe()
    Top15['ratio'] = Top15['Self-citations']/Top15['Citations']
    max_ratio = Top15['ratio'].max()
    country = Top15[Top15['ratio'] == Top15['ratio'].max()].index.tolist()[0]
    return (country, max_ratio)


('China', 0.68931261793894216)

In [608]:
def population(): #returns the population and returns the 3rd most populas country
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    Top15 = Top15.sort(['Population'], ascending = False)
    return Top15[2:3].index.tolist()[0]


/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


'United States'

In [609]:
def correlation(): #function estimates number of citable documents per person and returns the correlation between the number of citable document and energy supply per capita
    import numpy as np
    Top15 = merged_dataframe()
    Top15['Population'] = Top15['Energy Supply']/ Top15['Energy Supply per Capita']
    Top15['citable per person'] = Top15['Citable documents']/Top15['Population']
    Top15['citable per person'] = (Top15['citable per person'])
    Top15['citable per person'] = Top15['citable per person'].astype(float)
    Top15['Energy Supply per Capita'] = Top15['Energy Supply per Capita'].astype(float)
    corr = (Top15['citable per person']).corr(Top15['Energy Supply per Capita'])
    return corr


0.79400104354429424

In [610]:
def plot9(): #function plots the energy supply per capita and citable documents
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = merged_dataframe()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [611]:
def renewable_above_median(): #returns 1 for all the countries whose %renewable value is above the median for all the top 15 countries and 0 if it is below
    Top15 = merged_dataframe()
    Top15['median'] = Top15['% Renewable'].median()
    Top15['binary'] = 1
    id = Top15[Top15['% Renewable'] < Top15['median']].index
    Top15['binary'].loc[id] = 0
    HighRenew = Top15['binary']
    return HighRenew 


/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: binary, dtype: int64

In [639]:
def sample_size(): # dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.
    Top15 = merged_dataframe()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top15['Continent'] = pd.Series(ContinentDict)
    Top15['PopEst'] = (Top15['Energy Supply'] / Top15['Energy Supply per Capita']).astype(float)
    Top15 = Top15.set_index(['Continent'])
    Top15_1 = Top15.groupby(Top15.index).count()
    Top15_1['size'] = Top15_1.values[:,:1]
    Top15_2 = Top15.groupby(Top15.index).sum()
    Top15_1['sum'] = Top15_2['PopEst']
    Top15_3 = Top15.groupby(Top15.index).mean()
    Top15_1['mean'] = Top15_3['PopEst']
    Top15_1 = Top15_1.drop(Top15_1.columns[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]], axis = 1)
    Top15_4 = Top15.groupby(Top15.index).std()
    Top15_1['std'] = Top15_4['PopEst']
    return Top15_1


In [613]:
def bins(): #cuts % renewable into 5 bins and groups dataframe by continent and %renewable and retuns the series with multiindex of continent and bins for %renewable
    import pandas as pd
    import numpy as np
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top15 = merged_dataframe()
    Top15 = Top15.reset_index()
    Top15['Continent'] = [ContinentDict[country] for country in Top15['Country']]
    Top15['bins'] = pd.cut(Top15['% Renewable'],5)
    ans = Top15.groupby(['Continent', 'bins']).size()
    return ans


Continent      bins            
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
dtype: int64

In [635]:
def pop_to_string(): #converts the Population Estimate series to a string with thousands separator (using commas)
    Top15 = merged_dataframe()
    Top15['PopEst'] = (Top15['Energy Supply'] / Top15['Energy Supply per Capita']).astype(float)
    return Top15['PopEst'].apply(lambda x: '{0:,}'.format(x))


Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: PopEst, dtype: object